In [151]:
import re
from pathlib import Path
from typing import List, Dict

def _count_words(text: str) -> int:
    # Đếm "word" theo ký tự chữ/số Unicode (hợp với tiếng Việt có dấu)
    return len(re.findall(r"\b\w+\b", text, flags=re.UNICODE))

def chunk_markdown_folder_to_dicts(
    folder_path: str,
    glob_pattern: str = "**/*.md",
    min_h3_words: int = 100,
    min_h4_words: int = 100,
    min_h5_words: int = 100,
) -> List[Dict[str, str]]:
    folder = Path(folder_path)
    md_files = sorted(folder.glob(glob_pattern))
    chunks: List[Dict[str, str]] = []

    h1_re = re.compile(r"^\s*#\s+(?P<title>.+?)\s*$", re.MULTILINE)
    h2_re = re.compile(r"^\s*##\s+(?P<title>.+?)\s*$", re.MULTILINE)
    h3_re = re.compile(r"^\s*###\s+(?P<title>.+?)\s*$", re.MULTILINE)
    h4_re = re.compile(r"^\s*####\s+(?P<title>.+?)\s*$", re.MULTILINE)
    h5_re = re.compile(r"^\s*#####\s+(?P<title>.+?)\s*$", re.MULTILINE)

    for fp in md_files:
        text = fp.read_text(encoding="utf-8", errors="ignore").replace("\r\n", "\n")

        m1 = h1_re.search(text)
        if not m1:
            continue

        subject = m1.group("title").strip()
        subject_name = f"{subject.lower().replace('notes', '').replace('#', '').strip()}"

        h2_matches = list(h2_re.finditer(text))
        if not h2_matches:
            chunks.append({
                "subject": subject_name,
                "title": f"{subject} - (no section)",
                "content": f"# {subject}\n"
            })
            continue

        for i, m2 in enumerate(h2_matches):
            h2_title = m2.group("title").strip()
            h2_start = m2.start()
            h2_end = h2_matches[i+1].start() if i + 1 < len(h2_matches) else len(text)
            h2_block = text[h2_start:h2_end].strip("\n")

            # phần thân của H2 (bỏ dòng "## ...")
            h2_body = re.sub(r"^\s*##\s+.*\n?", "", h2_block, count=1).lstrip("\n")

            def emit_h2_chunk():
                content = f"# {subject}\n## {h2_title}\n"
                if h2_body.strip():
                    content += h2_body.rstrip() + "\n"
                chunks.append({
                    "subject": subject_name,
                    "title": f"{subject} - {h2_title}",
                    "content": content
                })

            # tìm H3 trong H2 body
            h3_matches = list(h3_re.finditer(h2_body))
            if not h3_matches:
                emit_h2_chunk()
                continue

            # Parse H3 -> (option) H4 -> (option) H5
            h3_entries = []
            for j, m3 in enumerate(h3_matches):
                h3_title = m3.group("title").strip()
                s3 = m3.start()
                e3 = h3_matches[j+1].start() if j + 1 < len(h3_matches) else len(h2_body)
                h3_block = h2_body[s3:e3].strip("\n")

                # body của H3 (bỏ dòng "### ...")
                h3_body = re.sub(r"^\s*###\s+.*\n?", "", h3_block, count=1).lstrip("\n")
                h3_wc = _count_words(h3_body)

                h4_matches = list(h4_re.finditer(h3_body))
                can_split_h4 = False
                h4_entries = []

                if h4_matches:
                    all_h4_big_enough = True
                    for k, m4 in enumerate(h4_matches):
                        h4_title = m4.group("title").strip()
                        s4 = m4.start()
                        e4 = h4_matches[k+1].start() if k + 1 < len(h4_matches) else len(h3_body)
                        h4_block = h3_body[s4:e4].strip("\n")

                        # body của H4 (bỏ dòng "#### ...")
                        h4_body = re.sub(r"^\s*####\s+.*\n?", "", h4_block, count=1).lstrip("\n")
                        h4_wc = _count_words(h4_body)
                        if h4_wc < min_h4_words:
                            all_h4_big_enough = False

                        # parse H5 trong H4 body
                        h5_matches = list(h5_re.finditer(h4_body))
                        can_split_h5 = False
                        h5_blocks = []

                        if h5_matches:
                            all_h5_big_enough = True
                            for t, m5 in enumerate(h5_matches):
                                h5_title = m5.group("title").strip()
                                s5 = m5.start()
                                e5 = h5_matches[t+1].start() if t + 1 < len(h5_matches) else len(h4_body)
                                h5_block = h4_body[s5:e5].strip("\n")

                                # body của H5 (bỏ dòng "##### ...")
                                h5_body = re.sub(r"^\s*#####\s+.*\n?", "", h5_block, count=1).lstrip("\n")
                                h5_wc = _count_words(h5_body)
                                if h5_wc < min_h5_words:
                                    all_h5_big_enough = False
                                h5_blocks.append((h5_title, h5_body))

                            can_split_h5 = all_h5_big_enough

                        h4_entries.append({
                            "h4_title": h4_title,
                            "h4_body": h4_body,      # giữ nguyên (có thể chứa H5 headings)
                            "h4_wc": h4_wc,
                            "can_split_h5": can_split_h5,
                            "h5_blocks": h5_blocks,
                        })

                    can_split_h4 = all_h4_big_enough

                h3_entries.append({
                    "h3_title": h3_title,
                    "h3_body": h3_body,      # giữ nguyên (có thể chứa H4/H5 headings)
                    "h3_wc": h3_wc,
                    "can_split_h4": can_split_h4,
                    "h4_entries": h4_entries,
                })

            # Nếu có bất kỳ H3 nào không split H4 được và cũng < min_h3_words -> fallback về H2
            if any((not e["can_split_h4"]) and (e["h3_wc"] < min_h3_words) for e in h3_entries):
                emit_h2_chunk()
                continue

            # Emit: ưu tiên H5 > H4 > H3
            for e3 in h3_entries:
                h3_title = e3["h3_title"]

                if e3["can_split_h4"]:
                    for e4 in e3["h4_entries"]:
                        h4_title = e4["h4_title"]

                        if e4["can_split_h5"]:
                            # chunk theo H5
                            for h5_title, h5_body in e4["h5_blocks"]:
                                content = (
                                    f"# {subject}\n"
                                    f"## {h2_title}\n"
                                    f"### {h3_title}\n"
                                    f"#### {h4_title}\n"
                                    f"##### {h5_title}\n"
                                )
                                if h5_body.strip():
                                    content += h5_body.rstrip() + "\n"
                                chunks.append({
                                    "subject": subject_name,
                                    "title": f"{subject} - {h2_title} - {h3_title} - {h4_title} - {h5_title}",
                                    "content": content
                                })
                        else:
                            # fallback về H4
                            content = (
                                f"# {subject}\n"
                                f"## {h2_title}\n"
                                f"### {h3_title}\n"
                                f"#### {h4_title}\n"
                            )
                            if e4["h4_body"].strip():
                                content += e4["h4_body"].rstrip() + "\n"
                            chunks.append({
                                "subject": subject_name,
                                "title": f"{subject} - {h2_title} - {h3_title} - {h4_title}",
                                "content": content
                            })
                else:
                    # fallback về H3
                    content = f"# {subject}\n## {h2_title}\n### {h3_title}\n"
                    if e3["h3_body"].strip():
                        content += e3["h3_body"].rstrip() + "\n"
                    chunks.append({
                        "subject": subject_name,
                        "title": f"{subject} - {h2_title} - {h3_title}",
                        "content": content
                    })

    return chunks


# === Dùng thử ===
folder_path = "./corpus"
docs = chunk_markdown_folder_to_dicts(
    folder_path,
    min_h3_words=70,
    min_h4_words=70,
    min_h5_words=70,
)

print("Total chunks:", len(docs))
print(docs[0]["title"])
print(docs[0]["content"][:600])


Total chunks: 259
Notes Kinh tế vĩ mô - GDP, GNP và các khái niệm “thu nhập quốc gia”
# Notes Kinh tế vĩ mô
## GDP, GNP và các khái niệm “thu nhập quốc gia”
- **Phần/Mục:** IV — Đo lường thu nhập quốc gia
- **Định nghĩa lõi:**
  - **GDP (Gross Domestic Product):** giá trị thị trường của **hàng hóa & dịch vụ cuối cùng** sản xuất **trong lãnh thổ** một quốc gia trong một thời kỳ.
  - **GNP (Gross National Product):** giá trị sản xuất bởi **yếu tố sở hữu bởi công dân/quốc gia** (national), bất kể ở đâu.
  - **NDP/NNP:** GDP/GNP **trừ khấu hao** (depreciation).
- **Công thức/Quan hệ:**
  - GDP = Σ (Giá * Số lượng) của **final goods/services**.
  - GNP = GDP + **NFIA** (Net Factor I


In [152]:
import json
json.dump(docs, open('./corpus/chunks.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=2)

In [153]:
len(docs)

259

In [154]:
sum([len(d['content'].strip()) for d in docs])

220324

In [155]:
len(set([d['title'].lower().strip() for d in docs]))

259

In [156]:
print('\n'.join(set(f"{d['subject']}: {max(len(dd['content'].strip().split())for dd in docs if dd['subject'] == d['subject'])}" for d in docs)))

vật lý đại cương ii: 326
hệ tuyến tính & biến đổi laplace: 272
vật lý đại cương iii: 369
pháp luật đại cương: 565
sinh học: 290
xác suất – thống kê: 789
toán rời rạc: 572
tư tưởng hồ chí minh: 301
giải tích nhiều biến: 304
đại số tuyến tính: 198
vật lý đại cương i: 117
kinh tế vĩ mô: 312
giải tích một biến: 284
kinh tế vi mô: 316
hóa học đại cương: 190
lịch sử đảng cộng sản việt nam: 235
